# Fusion

Submission 2:
    - Tremor : Gradient Boosting Regression
    - Dyskinesia : Gradient Boosting Regression
    
Submission 4: 
    - Dyskinesia : Average of predictions 

In [2]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

%load_ext autoreload
%autoreload 2

from transform_data import *
from create_graphs import *
from beatPDivec.default_data.v2_auto.local.get_final_scores_accuracy import *

In [3]:
# Path to where you want the CSV files created 
sDirOut = "/export/b19/mpgill/BeatPD_predictions_tryingout/"
# sDirOut = "<your-out-path>"

# Path to the folder containing the GitHub repository of BeatPD-CLSP-JHU
sPathGithub = os.getcwd()

# Path to the folder containing the kaldi/ folder
# sPathKaldi = "<path-to-kaldi>/"
sPathKaldi = "/export/b19/mpgill/"

sPathData = "/home/sjoshi/codes/python/BeatPD/data/BeatPD/"

# Fusion for REAL-PD sensors

For REAL-PD, we have these rules to create the predictions:
    - If we have all 3 predictions, we do the average of the two closest 
    - If there are only two measurements, we do the average of both 
    - If only one prediction, we take that one
    
The following code in this Notebook performs this merge to go from three subtypes of REAL-PD predictions to only one CSV file. 

In [5]:
def real_average_fusion(lFilesPred, sDirOut, sTypeLabel):
    iNumFiles=len(lFilesPred)
    vPredIter=np.zeros((1,iNumFiles))
    mPredictions=np.zeros((1,iNumFiles))
    lID=[] # measurement_id 
    lDicts=[] 
    vParID=[] # participant ID

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (acoustic, w-embed)
        lDicts.append(dPred)
   
    # Append all unique measurement ID as the lead
    all_unique_measurements = set(list(lDicts[0].keys()) + list(lDicts[1].keys()) + list(lDicts[2].keys()))

    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in all_unique_measurements:
        if k!='measurement_id':
            lID.append(k) # measurement_id 
            fPred=lDicts[0].get(k)
            if fPred:
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            else:
                vPredIter[0,0]=None
            # Go through the second and next predictions files
            for j in range(1, iNumFiles):
                # fPred will contain the prediction of the predictions file we're going through
                fPred=lDicts[j].get(k)
                if fPred: # if we found a prediction
                    # we add it in the array
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    #print([str(j) + 'Unknown key:' + k])
                    vPredIter[0,j]=None#float(np.asarray(lDicts[0][k]))
            if bEnter==1:
                # if it's the first prediction file, we initialize it
                mPredictions=vPredIter.copy() # mPredictions initialization.
                bEnter=0
            else:
                #[a, b, c]
                dist_a_b = abs(vPredIter[0,1] - vPredIter[0,0])
                dist_c_b = abs(vPredIter[0,2] - vPredIter[0,1])
                dist_c_a = abs(vPredIter[0,2] - vPredIter[0,0])

                min_value = min(np.array([dist_a_b, dist_c_b, dist_c_a]))
                
                if min_value == dist_a_b:
                    vPredIter[0,2] = None
                elif min_value == dist_c_b:
                    vPredIter[0,0] = None
                elif min_value == dist_c_a:
                    vPredIter[0,1] = None
                    
                # We add the new predictions as a new column if it's not the first file we're going through
                mPredictions=np.append(mPredictions,vPredIter,axis=0)

    # For REAL-PD, we have these rules to create the predictions:
    # If we have all 3 predictions, we do the average of the two closest 
    # If there are only two measurements, we do the average 
    
#     print(mPredictions)
    
    vAverage=np.nanmean(mPredictions,axis=1)
    
    lID, vPrediction=zip(*sorted(zip(lID, vAverage)))
    df = pd.DataFrame({'measurement_id': lID, sTypeLabel:vPrediction})
    df.to_csv(sDirOut+'submissionRealPD'+sTypeLabel+'.csv', index=False)
    print('Submission file was created: '+sDirOut+"submissionRealPD"+sTypeLabel+".csv")
    
    return vAverage

## ON/OFF

In [193]:
sPhoneAccPred=sPathGithub+'/tsfresh/submit/submission/phoneacc_on_off_new.csv'
sWatchAccPred=sPathGithub+'/tsfresh/submit/submission/watchacc_on_off_new.csv'
sWatchGyrPred=sPathGithub+'/tsfresh/submit/submission/watchgyr_on_off_new.csv'

lFilesPred=[sPhoneAccPred,sWatchAccPred,sWatchGyrPred]

sTypeLabel = "on_off"

# Name of the CSV file to be created with the average 
fileName ="on_off_real"

real_average_fusion(lFilesPred, sDirOut, sTypeLabel)

Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionRealPDon_off.csv


## Tremor

In [194]:
sPhoneAccPred=sPathGithub+'/tsfresh/submit/submission/phoneacc.tremor.csv'
sWatchAccPred=sPathGithub+'/tsfresh/submit/submission/watchacc.tremor.csv'
sWatchGyrPred=sPathGithub+'/tsfresh/submit/submission/watchgyr.tremor.csv'

lFilesPred=[sPhoneAccPred,sWatchAccPred,sWatchGyrPred]

sTypeLabel = "tremor"

# Name of the CSV file to be created with the average 
fileName ="tremor_real"

real_average_fusion(lFilesPred, sDirOut, sTypeLabel)

Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionRealPDtremor.csv


array([0.37931034, 0.65416554, 2.18682595, 0.37931034, 0.14303959,
       0.27248547, 2.02380952, 0.43902439, 0.2993007 , 0.13793103,
       0.40129212, 0.14303959, 0.32298894, 0.41187739, 0.41666667,
       0.68810978, 0.7608293 , 0.21676815, 2.13043478, 0.33974428,
       0.41666667, 0.29090909, 0.13793103, 0.44444444, 0.13793103,
       0.39955193, 0.57267669, 2.0053676 , 0.29090909, 0.2993007 ,
       0.37931034, 0.13793103, 0.35403822, 0.81333286, 2.02336013,
       0.30956461, 0.39560964, 0.13793103, 0.30769231, 0.13793103,
       0.45444199, 0.79937801, 2.02380952, 2.13043478, 0.2993007 ,
       1.99466338, 2.33280802, 2.13043478, 0.48426454, 0.13793103,
       0.41666667, 2.18454935, 0.19007548, 0.41666667, 2.16643537,
       2.01350806, 2.38435479, 0.41666667, 2.02380952, 2.04134894,
       2.02380952, 2.39958423, 0.43902439, 0.43646137, 0.42430604,
       2.13043478, 0.96504882, 0.30129155, 2.20308919, 0.29090909,
       0.34564532, 0.45395537, 0.30769231, 2.33909456, 2.48840

## Dyskinesia

In [195]:
sPhoneAccPred=sPathGithub+'/tsfresh/submit/submission/phoneacc.dyskinesia.csv'
sWatchAccPred=sPathGithub+'/tsfresh/submit/submission/watchacc.dyskinesia.csv'
sWatchGyrPred=sPathGithub+'/tsfresh/submit/submission/watchgyr.dyskinesia.csv'

lFilesPred=[sPhoneAccPred,sWatchAccPred,sWatchGyrPred]

sTypeLabel = "dyskinesia"

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_real"

real_average_fusion(lFilesPred, sDirOut, sTypeLabel)

Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionRealPDdyskinesia.csv


array([0.93285683, 0.28006747, 0.17857143, 0.95436689, 0.90929966,
       0.32372102, 0.89552239, 0.20036508, 0.18518519, 0.89552239,
       1.00760241, 0.18518519, 0.26308097, 0.89929588, 0.75281125,
       0.92863867, 0.79298685, 0.90929966, 0.2064634 , 0.28883904,
       0.64864865, 0.2008349 , 0.24714354, 0.91559994, 0.64864865,
       0.19259259, 0.18518519, 0.1925402 , 0.62068966, 0.26079421,
       0.9305576 , 0.21491919, 0.32213041, 0.18518519, 0.17857143,
       0.7692336 , 0.92585684, 0.2182692 , 0.34650603, 0.25443814,
       0.54594984, 0.23603459, 0.62068966, 0.62068966, 0.18695809,
       0.64534303, 0.42934678, 0.22498779, 0.18518519, 0.20650673,
       0.6146469 , 0.97777942, 0.39764001, 0.62068966, 0.68154659,
       0.62068966, 0.30773422, 0.31217173, 0.89552239, 0.71826372,
       0.65007382, 1.05016382, 0.9542592 , 0.23760092, 0.27813159,
       0.68247064, 1.00831413, 1.02909242, 0.80784821, 0.16883117,
       0.18903047, 0.18518519, 0.94013955, 0.17957755, 0.24756

## Submission 4 - Average of predictions for Approach 1 and 2 - CIS-PD 

In [8]:
def average_fusion(lFilesPred, sDirOut, fileName, bRound, sFileLabels, sTypeLabel, bTestSubset=False):
    """
    Performs Fusion for CIS-PD Approach 1 (tsfresh + xgboost) and Approach 2 (SVR Per Patient)
    
    Keyword arguments:
    TODO
    """
    
    # Read file labels (true labels)
    if not bTestSubset:
        print("sFileLabels : ", sFileLabels)
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dID = {rows[0]:rows[1] for rows in reader} #participant ID
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dTrem={rows[0]:rows[4] for rows in reader} #tremor label

        #label selection
        if sTypeLabel=='on_off':
            dLabels=dOnOff
        elif sTypeLabel=='tremor':
            dLabels=dTrem
        elif sTypeLabel=='dyskinesia':
            dLabels=dDys
        else:
            print('sTypeLabel undefined')
        
    iNumFiles=len(lFilesPred)
    print('Number of analyzed file inputs: '+ str(iNumFiles))
    vPredIter=np.zeros((1,iNumFiles))
    mPredictions=np.zeros((1,iNumFiles))
    lID=[] # measurement_id 
    lDicts=[] 
    vLabels = [] # true label
    vParID=[] # participant ID

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (acoustic, w-embed)
        lDicts.append(dPred)
   
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            lID.append(k) # measurement_id 
            if not bTestSubset:
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            # Go through the second and next predictions files
            for j in range(1, iNumFiles):
                # fPred will contain the prediction of the predictions file we're going through
                fPred=lDicts[j].get(k)
                if fPred: # if we found a prediction
                    # we add it in the array
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
            if bEnter==1:
                # if it's the first prediction file, we initialize it
                mPredictions=vPredIter.copy() # mPredictions initialization.
                bEnter=0
            else:
                # We add the new predictions as a new column if it's not the first file we're going through
                mPredictions=np.append(mPredictions,vPredIter,axis=0)

    vAverage=np.mean(mPredictions,axis=1)
    if bRound==1:
        vPrediction=np.round(vAverage)
    else:
        vPrediction=vAverage
    
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]

    lID, vPrediction=zip(*sorted(zip(lID, vPrediction)))
    df = pd.DataFrame({'measurement_id': lID, sTypeLabel:vPrediction})
    df.to_csv(sDirOut+'submissionCisPD'+sTypeLabel+'.csv', index=False)
    print('Submission file was created: '+sDirOut+"submissionCisPD"+sTypeLabel+".csv")
    
    return vRes1, vRes2, vAverage, vParID, vLabels

### Dyskinesia

### Get predictions file on test subset 

In [200]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/submission/cis-pd.dyskinesia.perpatient.csv'

# SVR predictions files on test kfolds 
sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0

# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel, True)


Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDdyskinesia.csv


# Merge CIS-PD and REAL-PD predictions in one CSV file 

In [208]:
from os import chdir
from glob import glob
import pandas as pdlib

def produceOneCSV(list_of_files, file_out):
    """
    Produce a single CSV after combining all files

    Source: https://www.techbeamers.com/pandas-merge-csv-files/
    """
    # Consolidate all CSV files into one object
    result_obj = pdlib.concat([pdlib.read_csv(file) for file in list_of_files])
    # Convert the above object into a csv file and export
    result_obj.to_csv(file_out, index=False, encoding="utf-8")
    print('File was created: ', file_out)

## ON/OFF

In [209]:
# Merge tsfresh/.../cis-pd.on_off_new.csv and sDirOut+submissionRealPDon_off.csv

sCisFile = sPathGithub+'/tsfresh/submit/submission/cis-pd.on_off_new.csv'
sRealFile = sDirOut+'submissionRealPDon_off.csv'

list_of_files = [sCisFile, sRealFile]
file_out = sDirOut+"/cis-real.on_off.csv"
produceOneCSV(list_of_files, file_out)

File was created:  /export/b19/mpgill/BeatPD_predictions_tryingout//cis-real.on_off.csv


## Tremor

In [210]:
# Merge /tsfresh/.../cis-pd.tremor.csv and submissionRealPDtremor.csv 

sCisFile = sPathGithub+'/tsfresh/submit/submission/cis-pd.tremor.csv'
sRealFile = sDirOut+'submissionRealPDtremor.csv'

list_of_files = [sCisFile, sRealFile]
file_out = sDirOut+"/cis-real.tremor.csv"
produceOneCSV(list_of_files, file_out)

File was created:  /export/b19/mpgill/BeatPD_predictions_tryingout//cis-real.tremor.csv


## Dyskinesia

In [211]:
# Merge submissionCisPDdyskinesia.csv and submissionRealPDdyskinesia.csv

sCisFile = sDirOut+"submissionCisPDdyskinesia.csv"
sRealFile = sDirOut+'submissionRealPDdyskinesia.csv'

list_of_files = [sCisFile, sRealFile]
file_out = sDirOut+"/cis-real.dyskinesia.csv"
produceOneCSV(list_of_files, file_out)

File was created:  /export/b19/mpgill/BeatPD_predictions_tryingout//cis-real.dyskinesia.csv


### Example on test folds

In [ ]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/submission/cis-pd.dyskinesia.perpatient.csv'

# SVR predictions files on test kfolds 
sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

### Tremor - Test Folds - CIS-PD - Gridsearch + global norm

In [8]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_cis-pd_tremor.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="tremor_test"

sTypeLabel = "tremor"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDtremor.csv
--- MSEscore ---
Final score :  0.4425578721059196
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.4425578721059196
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.4425578721059196
Overall MSE Fusion - average :  None


### Dyskinesia - Gridsearch + global norm

In [9]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_cis-pd_dyskinesia.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDdyskinesia.csv
--- MSEscore ---
Final score :  0.4896636009359452
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.4896636009359452
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.4896636009359452
Overall MSE Fusion - average :  None


### On/Off - Gridsearch + global norm

In [10]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_cis-pd_on_off.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="on_off_teeeest"

sTypeLabel = "on_off"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDon_off.csv
--- MSEscore ---
Final score :  1.156679002684981
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  1.156679002684981
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  1.156679002684981
Overall MSE Fusion - average :  None


In [24]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_lambda_0.3_cis-pd_on_off.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_teeerrrest"

sTypeLabel = "on_off"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDon_off.csv
--- MSEscore ---
Final score :  1.1948072541641777
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  1.1948072541641777
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  1.1948072541641777
Overall MSE Fusion - average :  None


In [9]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_rf_cis-pd_on_off.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_teeerrrest"

sTypeLabel = "on_off"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDon_off.csv
--- MSEscore ---
Final score :  1.1750855614237372
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  1.1750855614237372
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  1.1750855614237372
Overall MSE Fusion - average :  None


In [10]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_rf_cis-pd_tremor.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_teeerrrest"

sTypeLabel = "tremor"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDtremor.csv
--- MSEscore ---
Final score :  0.4503645126275858
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.4503645126275858
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.4503645126275858
Overall MSE Fusion - average :  None


In [26]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_rf_cis-pd_tremor.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_teeerrrest"

sTypeLabel = "tremor"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDtremor.csv
--- MSEscore ---
Final score :  0.45107914133573473
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.45107914133573473
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.45107914133573473
Overall MSE Fusion - average :  None


In [20]:
# tsfresh predictions file on test kfolds 
sFilePred1=sPathGithub+'/tsfresh/submit/kfold_prediction_cis-pd_dyskinesia_lamb_1.0_2.csv'

# SVR predictions files on test kfolds 
# sFilePred2=sPathKaldi+'/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred1]

# Path to labels on CIS-PD 
sFileLabels=sPathData+'/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dyskinesia_teeerrrest"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))

sFileLabels :  /home/sjoshi/codes/python/BeatPD/data/BeatPD//cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
Number of analyzed file inputs: 2
Submission file was created: /export/b19/mpgill/BeatPD_predictions_tryingout/submissionCisPDdyskinesia.csv
--- MSEscore ---
Final score :  0.5132973963760121
Overall MSE Classif. 1 - tsfresh:  None
--- MSEscore ---
Final score :  0.5132973963760121
Overall MSE Classif. 2 - ivec:  None
--- MSEscore ---
Final score :  0.5132973963760121
Overall MSE Fusion - average :  None
